In [22]:
from sklearn.neighbors import BallTree
from skbio import OrdinationResults
import pandas as pd
import numpy as np
import qgrid
from knn import KNN
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

In [23]:
qgrid.enable(dataframe=True)
qgrid.set_grid_option('maxVisibleRows', 10)
qgrid.set_grid_option('forceFitColumns', False)
qgrid.set_grid_option('defaultColumnWidth', 120)
#x = pd.DataFrame(np.random.random((100, 100)))

In [24]:
pcoa = OrdinationResults.read('data/generalized-unifrac-0.1-pcoa/ordination.txt')
metadata = pd.read_table('healthy_metadata.txt', 
                         index_col=0,dtype=str,
                         na_values=['Unspecified', 'NaN'])
metadata = metadata.iloc[:, 42:]

In [25]:
m = (pcoa.samples**2).sum(axis=0)
coords = pcoa.samples.loc[:, pcoa.proportion_explained.cumsum() < 0.99]
metadata = metadata.sample(frac=1) 
#metadata['subset_healthy'] = np.logical_or( 
#    metadata.subset_healthy == 'Yes',
#    metadata.subset_healthy == 'True'
#)    
coords = coords.reindex(index=metadata.index)

In [38]:
obs_md = metadata.iloc[1000:] 
obs_md = obs_md.loc[obs_md.healthy=='True']
svm = OneClassSVM(nu=0.5) 
#svm = IsolationForest(contamination=0.5, n_estimators=100)
model = svm.fit(coords.loc[obs_md.index]) 

In [39]:
pred = model.predict(coords.iloc[:1000])

In [40]:
pd.DataFrame({'healthy': metadata.iloc[:1000].healthy=='True', 
              'pred': pred==1}).astype(np.bool)

A Jupyter Widget

In [41]:
from sklearn.metrics import roc_auc_score
y_obs = metadata.iloc[:1000].healthy=='True'
y_pred = pred==1
#y_obs = obs_md.autoimmune != 'I do not have this condition'
roc_auc_score(y_pred, y_obs, average='weighted')

0.54021839808901673